In [15]:
import os
from api import API_KEYS

os.environ["OPENAI_API_KEY"] = API_KEYS["OPENAI"]
os.environ["HUGGINGFACEHUB_API_TOKEN"] = API_KEYS["HUGGINFACE"]

## API CALLING TEST

### 프롬포트 생성

- PromptTemplate은 LLM에 문장을 전달하기 전에 문장 구성을 편리하게 만들어주는 역할
- LLM이 어떤 문장을 만들어야 하는지를 알려주는 역할을 수행

In [ ]:
from langchain import PromptTemplate

template = "{product}를 홍보하기 위한 좋은 문구를 추천해줘?"

prompt = PromptTemplate(
    input_variables=["product"],
    template=template,
)

prompt.format(product="카메라")

'카메라를 홍보하기 위한 좋은 문구를 추천해줘?'

### OPENAI

In [ ]:
from langchain.chat_models import ChatOpenAI

llm1 = ChatOpenAI(temperature=0,
                  model_name='gpt-4')

prompt = "상범이는 거북이를 키우고 있습니다. 상범이가 키우고 있는 동물은?"

print(llm1.predict(prompt))

/opt/anaconda3/envs/llm/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


거북이입니다.


In [ ]:
from langchain_openai import ChatOpenAI

llm1 = ChatOpenAI(temperature=0,
                  model_name='gpt-4')

prompt = "상범이는 거북이를 키우고 있습니다. 상범이가 키우고 있는 동물은?"

print(llm1.invoke(prompt))

content='거북이입니다.' response_metadata={'token_usage': {'completion_tokens': 6, 'prompt_tokens': 42, 'total_tokens': 48}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-ea91daf9-1891-405f-8622-a39ab83a1a33-0' usage_metadata={'input_tokens': 42, 'output_tokens': 6, 'total_tokens': 48}


### HUGGING FACE
- 시간마다 request할 수 있는 양의 한계가 있음
- google/flan-t5-xxl 모델 사용시, 바로 request 에러

In [ ]:
from langchain import HuggingFaceHub

llm2 = HuggingFaceHub(repo_id = "google/flan-t5-small", 
                      model_kwargs={"temperature":0, "max_depth":512})

completion = llm2(prompt)
print(completion)

### Compare

In [ ]:
from langchain.model_laboratory import ModelLaboratory
model_lab = ModelLaboratory.from_llms([llm1, llm2])
model_lab.compare("대한민국의 가을은 몇 월부터 몇 월까지야?")

## Output Parser

In [ ]:
from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain.prompts import PromptTemplate

from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0,
                 max_tokens=2048,
                 model_name='gpt-4',
                 )

output_parser = CommaSeparatedListOutputParser()
format_instructions = output_parser.get_format_instructions()

prompt = PromptTemplate(
    template="7개의 팀을 보여줘 {subject}.format\n{format_instructions}",
    input_variables=["subject"],
    partial_variables={"format_instructions": format_instructions}
)

In [ ]:
query = "한국의 야구팀은?"

output = llm.predict(text=prompt.format(subject=query))

# 출력의 format 변경
parsed_results = output_parser.parse(output)
print(parsed_results)

['두산 베어스', '롯데 자이언츠', '삼성 라이온즈', 'SK 와이번스', 'KIA 타이거즈', 'LG 트윈스', 'NC 다이노스']


## PDF

In [ ]:
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("The_Adventures_of_Tom_Sawyer.pdf")
document = loader.load()
document[5].page_content[:5000]

'Chapter 1    The Fence \n \nTom Sawyer lived with his aunt because his mother and \nfather were dead. Tom didn’t like going to school, and he didn’t like working. He liked playing and having adventures. One Friday, he didn’t go to school—he went to the river. \nAunt Polly was angry. “You’re a bad boy!” she said. \n“Tomorrow you can’t play with your friends because you didn’t go to school today. Tomorrow you’re going to work for me. You can paint the fence.” \nSaturday morning, Tom was not happy, but he started to \npaint the fence. His friend Jim was in the street. \nTom asked him, “Do you want to paint?” \nJim said, “No, I can’t. I’m going to get water.” \nThen Ben came to Tom’s house. He watched Tom and \nsaid, “I’m going to swim today. You can’t swim because you’re working.” \nTom said, “This isn’t work. I like painting.” \n“Can I paint, too?” Ben asked. \n“No, you can’t,” Tom answered. “Aunt Polly asked me \nbecause I’m a very good painter.” \nBen said, “I’m a good painter, too. P

In [ ]:
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()
db = FAISS.from_documents(document, embeddings)

/opt/anaconda3/envs/llm/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA

llm = ChatOpenAI(temperature=0,
                 model_name = 'gpt-3.5-turbo',
                 )

retriever = db.as_retriever()

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever
)

query = "마을 무덤에 있던 남자를 죽인 사람은 누구니?"
result = qa({'query': query})
print(result['result'])

마을 무덤에 있던 남자를 죽인 사람은 Injun Joe입니다.
